## Data Fetching

In [1]:
import urllib
dls = "https://static-content.springer.com/esm/art%3A10.1038%2Fs41587-020-0684-z/MediaObjects/41587_2020_684_MOESM2_ESM.xlsx"
urllib.request.urlretrieve(dls, "covid_time_series.xls")




('covid_time_series.xls', <http.client.HTTPMessage at 0x1077baa58>)

## Data Cleaning/Transforming

In [61]:
import pandas as pd
covid_series = pd.read_excel("covid_time_series.xls").dropna().drop(["Date"])

/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:3111: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [62]:
covid_series = covid_series.drop(covid_series.index[[0,1]]).reset_index() 
covid_series.columns = ["Date","cases_specimen","percent","admits","cases_reported"]

In [63]:
covid_series = covid_series.drop(["percent","admits"],axis=1)

In [64]:
covid_series = covid_series.astype({'cases_specimen': 'float','cases_reported': 'float'})

In [65]:
covid_series["Date"] = pd.to_datetime(covid_series["Date"],format='%d-%b').apply(lambda dt: dt.replace(year=2020))

In [72]:
covid_series.head(7)

,Date,cases_specimen,cases_reported
0,2020-03-19,10.0,1.0
1,2020-03-20,5.0,2.0
2,2020-03-21,11.0,1.0
3,2020-03-22,10.0,4.0
4,2020-03-23,25.0,5.0
5,2020-03-24,39.0,13.0
6,2020-03-25,32.0,11.0


## Correlation Without Offset

### Pearson

In [69]:
import numpy as np
np.corrcoef(covid_series["cases_specimen"],covid_series["cases_reported"])

array([[1.        , 0.57310753],
       [0.57310753, 1.        ]])

### Spearsman

In [70]:
from scipy import stats
stats.spearmanr(covid_series["cases_specimen"], covid_series["cases_reported"])

SpearmanrResult(correlation=0.5986397613993998, pvalue=1.765657848906687e-08)

## Correlation With Offset cases report -5 days

In [75]:
covid_series_neg5 = covid_series.copy()
covid_series_neg5["cases_reported"] = covid_series_neg5["cases_reported"].shift(-5)
covid_series_neg5 = covid_series_neg5.dropna()

In [76]:
len(covid_series_neg5)

69